In [2]:
#!/usr/bin/env python3
import pandas as pd
from geopy.geocoders import Bing
import csv  
import json   
import warnings
warnings.filterwarnings('ignore')
bing_key = 'ginITT5ALcrOnA4aMoH0~_vGQBu-_tnKr6FYRj6U5nA~Ar9conblprRVoeaB4uH4PKMzcs0BfrUJ5LbspSyDn04UqCpCOfsm567DJjUw-PY6'

In [41]:
def get_geocode(row):
    geolocator = Bing(bing_key, timeout=7)
    location = geolocator.geocode(row)
    if location is None:
        print("NOT FOUND")
        return "NOT FOUND"
    else:
        if location.raw["address"]["countryRegion"] == "Chile":
            r = str(location.latitude) + ", " + str(location.longitude)
            print(r)
            return r
        else:
            print("MISFORMED")
            return "MISFORMED"
        
def geocode(tabla):
    if(tabla == 'LEED.csv'):
        a = pd.read_csv(tabla,sep=',')
        b = a.copy()

        c = b[(b['ProjectName'] != 'Confidential') & (b['Street'].notnull()) & (b['Country'] == 'CL')]

        c['Pais'] = 'Chile'
        c['direccion'] = c[['Street', 'City', 'State', 'Pais']].apply(lambda x: ', '.join(x), axis=1)    
        # Geocoding #
        c['Coordenadas'] = c['direccion'].apply(get_geocode)

        # Lat and Lon
        lat = lambda row: row.split(',')[0].strip()
        lon = lambda row: row.split(',')[-1].strip()
        c['lat'] = c['Coordenadas'].map(lat)
        c['lon'] = c['Coordenadas'].map(lon)
        c.drop(labels=['Coordenadas'],axis=1,inplace=True)

        # Save to CSV
        n = 'data/coor_' + tabla.split('.')[0] + '.csv'
        c.to_csv(n, sep=',', encoding='utf-8')
        print(tabla + ' Done! ')
        
    elif(tabla == 'CES.csv'):
        a = pd.read_csv(tabla,sep=',')
        b = a.copy()
        c = b[(b['Dirección'].notnull())]
        c['Pais'] = 'Chile'
        c['direccion'] = c[['Dirección', 'Comuna', 'UBICACION', 'Pais']].apply(lambda x: ', '.join(x), axis=1)
        
        # Geocoding #
        c['Coordenadas'] = c['direccion'].apply(get_geocode)

        # Lat and Lon
        lat = lambda row: row.split(',')[0].strip()
        lon = lambda row: row.split(',')[-1].strip()
        c['lat'] = c['Coordenadas'].map(lat)
        c['lon'] = c['Coordenadas'].map(lon)
        c.drop(labels=['Coordenadas'],axis=1,inplace=True)

        # Save to CSV
        n = 'data/coor_' + tabla.split('.')[0] + '.csv'
        c.to_csv(n, sep=',', encoding='utf-8')
        print(tabla + ' Done! ')  
        
#     # Open the CSV  
#     f = open(n, 'rU')  
#     # Change each fieldname to the appropriate field name. I know, so difficult.  
#     reader = csv.DictReader(f, fieldnames = ('ID','Isconfidential','ProjectName','Street','City','State','Zipcode','Country','LEEDSystemVersionDisplayName','Sistema','PointsAchieved','CertLevel','CertDate','Mes certificación','Año' 'certificación','IsCertified','OwnerTypes','GrossSqFoot','mt2','TotalPropArea','ProjectTypes','Categoría','OwnerOrganization','RegistrationDate','Mes registro','Año registro','Pais','direccion','lat','lon'))  
    
#     # Parse the CSV into JSON  
#     out = json.dumps( [ row for row in reader ] )  
#     print("JSON parsed!")  
    
#     # Save the JSON  
#     f = open('leed.json', 'w')  
#     f.write(out)  
    
#     print("JSON saved!") 

In [42]:
geocode('LEED.csv')
geocode('CES.csv')

-33.4136216, -70.6035582
-27.147939682006836, -109.42758178710938
-33.3840941, -70.5301508
-27.15257, -109.43649
-27.80712, -70.50437
-33.51499, -70.56055
-33.39032, -70.49858
-41.4711318, -72.9386701
-33.40414, -70.5703
-27.34547, -70.35142
-33.3773259, -70.5270803
-33.3918983, -70.6198008
-33.41541, -70.58387
-33.4575568, -70.6407847
-36.84621, -73.05298
-23.639450073242188, -70.3959732055664
-51.72413, -72.50217
-22.909570693969727, -68.19908905029297
-34.91088, -71.16179
-53.11221, -70.88237
-33.41784, -70.60021
-22.44726, -68.90392
-45.57935, -72.07679
-39.83943, -73.21797
-33.43165, -70.65285
-33.57771, -70.58242
-23.6091817, -70.3883942
-39.81094, -73.25228
-32.749481201171875, -70.72531127929688
-22.4503235, -68.9209045
-36.76654, -73.04932
-29.9126287, -71.2586572
-35.23200607299805, -71.21882629394531
-33.45137, -70.66168
-33.4653, -70.59808
-33.3495, -70.74479
-33.4162353, -70.6067506
-33.27974, -70.64115
-33.44279, -70.65116
-33.3545927, -70.7201658
-33.45137, -70.66168
-33

In [44]:
import csv, json
from geojson import Feature, FeatureCollection, Point

In [45]:
features = []
with open('data/coor_LEED.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for lat, lon in reader:
        lat, lon = map(float, (lat, lon))features.append(
            Feature(
                geometry = Point((lon, lat)),
                properties = {
                    'weather': 'weather',
                    'temp': 'temp'
                }
            )
        )

collection = FeatureCollection(features)
with open('data/coor_LEED.geojson', "w") as f:
    f.write('%s' % collection)

ValueError: too many values to unpack (expected 2)